In [1]:
!pip install sagemaker==1.72.0

     |████████████████████████████████| 297 kB 6.3 MB/s eta 0:00:01
  Created wheel for sagemaker: filename=sagemaker-1.72.0-py2.py3-none-any.whl size=386358 sha256=a2638ae6d9922f8ca9b0c949cc1f4292de93a5d176dabe988b2b8a8ee1588d82
  Stored in directory: /home/ec2-user/.cache/pip/wheels/c3/58/70/85faf4437568bfaa4c419937569ba1fe54d44c5db42406bbd7
Successfully built sagemaker
  Attempting uninstall: smdebug-rulesconfig
    Found existing installation: smdebug-rulesconfig 0.1.5
    Uninstalling smdebug-rulesconfig-0.1.5:
      Successfully uninstalled smdebug-rulesconfig-0.1.5
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.16.4.dev0
    Uninstalling sagemaker-2.16.4.dev0:
      Successfully uninstalled sagemaker-2.16.4.dev0
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


# Plagiarism Detection Model

Now that you've created training and test data, you are ready to define and train a model. Your goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features you provide the model.

This task will be broken down into a few discrete steps:

* Upload your data to S3.
* Define a binary classification model and a training script.
* Train your model and deploy it.
* Evaluate your deployed classifier and answer some questions about your approach.

To complete this notebook, you'll have to complete all given exercises and answer all the questions in this notebook.
> All your tasks will be clearly labeled **EXERCISE** and questions as **QUESTION**.

It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset.

---

## Load Data to S3

In the last notebook, you should have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, you can upload your locally stored data to S3.

Save your train and test `.csv` feature files, locally. To do this you can run the second notebook "2_Plagiarism_Feature_Engineering" in SageMaker or you can manually upload your files to this notebook using the upload icon in Jupyter Lab. Then you can upload local files to S3 by using `sagemaker_session.upload_data` and pointing directly to where the training data is saved.

In [2]:
import pandas as pd
import boto3
import sagemaker

In [3]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## EXERCISE: Upload your training data to S3

Specify the `data_dir` where you've saved your `train.csv` file. Decide on a descriptive `prefix` that defines where your data will be uploaded in the default S3 bucket. Finally, create a pointer to your training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) or previous SageMaker code examples.

You are expected to upload your entire directory. Later, the training script will only access the `train.csv` file.

In [4]:
# should be the name of directory you created to save your features data
data_dir = 'plagiarism_data'

# set prefix, a descriptive name for a directory  
prefix = 'plagiarism-detection'

# upload all data to S3
data_path  = sagemaker_session.upload_data(data_dir, bucket=bucket, key_prefix=prefix)

### Test cell

Test that your data has been successfully uploaded. The below cell prints out the items in your S3 bucket and will throw an error if it is empty. You should see the contents of your `data_dir` and perhaps some checkpoints. If you see any other files listed, then you may have some old model files that you can delete via the S3 console (though, additional files shouldn't affect the performance of model developed in this notebook).

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

plagiarism-detection/test.csv
plagiarism-detection/train.csv
Test passed!


---

# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

The type of model you create is up to you. For a binary classification task, you can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

It will be up to you to test out a variety of models and choose the best one. Your project will be graded on the accuracy of your final model. 
 
---

## EXERCISE: Complete a training script 

To implement a custom classifier, you'll need to complete a `train.py` script. You've been given the folders `source_sklearn` and `source_pytorch` which hold starting code for a custom Scikit-learn model and a PyTorch model, respectively. Each directory has a `train.py` training script. To complete this project **you only need to complete one of these scripts**; the script that is responsible for training your final model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model
Much of the training script code is provided for you. Almost all of your work will be done in the `if __name__ == '__main__':` section. To complete a `train.py` file, you will:
1. Import any extra libraries you need
2. Define any additional model training hyperparameters using `parser.add_argument`
2. Define a model in the `if __name__ == '__main__':` section
3. Train the model in that same section

Below, you can use `!pygmentize` to display an existing `train.py` file. Read through the code; all of your tasks are marked with `TODO` comments. 

**Note: If you choose to create a custom PyTorch model, you will be responsible for defining the model in the `model.py` file,** and a `predict.py` file is provided. If you choose to use Scikit-learn, you only need a `train.py` file; you may import a classifier from the `sklearn` library.

In [8]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_pytorch/train.py

import argparse
import json
import os
import pandas as pd
import torch
import torch.optim as optim
import torch.utils.data

# imports the model in model.py by name
from model import BinaryClassifier

def model_fn(model_dir):
    """Load the PyTorch model from the `model_dir` directory."""
    print("Loading model.")

    # First, load the parameters used to create the model.
    model_info = {}
    model_info_path = os.path.join(model_dir, 'model_info.pth')
    with open(model_info_path, 'rb') as f:
        model_info = torch.load(f)

    print("model_info: {}".format(model_info))

    # Determine the device and construct the model.
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BinaryClassifier(model_info['input_features'], model_info['hidden_dim'], model_info['output_dim'])

    # Load the stored model parameters.
    model_path = os.path.join(model_dir, 'model.pth')
    with open(model_path, 'rb') as f:
        model.load_state_dict(torch.load(

### Provided code

If you read the code above, you can see that the starter code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) or with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), click on the provided links.

---
# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function you specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `source_sklearn` OR `source_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

Note: For a PyTorch model, there is another optional argument **framework_version**, which you can set to the latest version of PyTorch, `1.0`.

## EXERCISE: Define a Scikit-learn or PyTorch estimator

To import your desired estimator, use one of the following lines:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

# Classifier-1: PyTorch Feedforward Neural Network

In [9]:
# your import and estimator code, here
from sagemaker.pytorch import PyTorch

# specify an output path
output_path = 's3://{}/{}'.format(bucket, prefix)

# instantiate a pytorch estimator
estimator = PyTorch(entry_point="train.py",
                    source_dir="source_pytorch",
                    role=role,
                    framework_version='1.4.0',
                    py_version='py3',
                    train_instance_count=1,
                    train_instance_type='ml.c4.xlarge',
                    output_path=output_path,
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'epochs': 10,
                        'hidden_dim': 20,
                        'batch-size': 10,
                        'input_features': 3,
                        'output_dim': 1,
                    })

## EXERCISE: Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

In [11]:
%%time

# Train your estimator on S3 training data
estimator.fit({'train': data_path})

'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-12-02 23:40:34 Starting - Starting the training job...
2020-12-02 23:40:36 Starting - Launching requested ML instances......
2020-12-02 23:41:56 Starting - Preparing the instances for training.........
2020-12-02 23:43:05 Downloading - Downloading input data...
2020-12-02 23:44:04 Training - Training image download completed. Training in progress.
2020-12-02 23:44:04 Uploading - Uploading generated training modelbash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-12-02 23:43:55,282 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-12-02 23:43:55,285 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-12-02 23:43:55,295 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-12-02 23:43:55,299 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-12-02 23:43:55,611 sagemaker-container

2020-12-02 23:44:01,372 sagemaker-containers INFO     Reporting training SUCCESS

2020-12-02 23:44:12 Completed - Training job completed
Training seconds: 67
Billable seconds: 67
CPU times: user 540 ms, sys: 40.4 ms, total: 581 ms
Wall time: 4min 12s


# Classifier-2: LinearLearner

In [100]:
# import LinearLearner
from sagemaker import LinearLearner

# define location to store model artifacts
cl2_prefix = 'plagiarism-detection-linearlearner'
output_path='s3://{}/{}/'.format(bucket, cl2_prefix)
# print('Training artifacts will be uploaded to: {}'.format(output_path))

# instantiate LinearLearner
linear = LinearLearner(role=role,
             train_instance_count=1,
             train_instance_type='ml.c4.xlarge',
             output_path=output_path,
             sagemaker_session=sagemaker_session,
             predictor_type='binary_classifier', # The type of predictor to learn
             epochs=15) # Default value. The maximum number of passes to make over the training data

In [21]:
import os
# read in the train csv file
train_df = pd.read_csv(os.path.join(data_dir, "train.csv"), header=None, names=None) #[num_files=70, num_columns=4]
train_labels = train_df.loc[:, 0].values #[num_files=70]
train_features = train_df.loc[:, 1:].values #[num_files=70, num_features=3]

# create RecordSet of training data
train_x_np = train_features.astype('float32')
train_y_np = train_labels.astype('float32')

formatted_train_data = linear.record_set(train=train_x_np, labels=train_y_np) #Build a RecordSet from a numpy ndarray matrix and label vector

In [22]:
%%time 
# train the estimator on formatted training data
linear.fit(formatted_train_data)

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-12-03 19:01:22 Starting - Starting the training job...
2020-12-03 19:01:24 Starting - Launching requested ML instances......
2020-12-03 19:02:29 Starting - Preparing the instances for training.........
2020-12-03 19:04:04 Downloading - Downloading input data...
2020-12-03 19:04:53 Training - Training image download completed. Training in progress.
2020-12-03 19:04:53 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
Running default environment configuration script
[12/03/2020 19:04:48 INFO 140670865348416] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_level': u'info', u'quantile': u'0


2020-12-03 19:05:01 Completed - Training job completed
Training seconds: 57
Billable seconds: 57
CPU times: user 607 ms, sys: 23.9 ms, total: 631 ms
Wall time: 4min 12s


# Classifier-3: Scikit-learn Naive Bayes

In [47]:
from sagemaker.sklearn.estimator import SKLearn

# define location to store model artifacts
cl3_prefix = 'plagiarism-detection-nb'
output_path='s3://{}/{}/'.format(bucket, cl3_prefix)
print('Training artifacts will be uploaded to: {}'.format(output_path))

sklearn_estimator = SKLearn('train.py',
                            train_instance_type='ml.c4.xlarge',
                            py_version='py3',
                            framework_version='0.23-1',
                            source_dir='source_sklearn',
                            role=role,
                            output_path=output_path,
                            sagemaker_session=sagemaker_session,)

sklearn_estimator.fit({'train': data_path})

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


Training artifacts will be uploaded to: s3://sagemaker-us-east-2-793227728067/plagiarism-detection-nb/
2020-12-03 20:42:47 Starting - Starting the training job...
2020-12-03 20:42:50 Starting - Launching requested ML instances......
2020-12-03 20:43:56 Starting - Preparing the instances for training...
2020-12-03 20:44:42 Downloading - Downloading input data...
2020-12-03 20:45:11 Training - Downloading the training image..2020-12-03 20:45:25,810 sagemaker-training-toolkit INFO     Imported framework sagemaker_sklearn_container.training
2020-12-03 20:45:25,812 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-12-03 20:45:25,821 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-12-03 20:45:26,164 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-12-03 20:45:26,826 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-12-03 20:45:26,837 sagemaker-t

## EXERCISE: Deploy the trained model

After training, deploy your model to create a `predictor`. If you're using a PyTorch model, you'll need to create a trained `PyTorchModel` that accepts the trained `<model>.model_data` as an input parameter and points to the provided `source_pytorch/predict.py` file as an entry point. 

To deploy a trained model, you'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type. It may help to refer to your previous exercise code to see which types of instances we used.

# Deploy Classifier-1

In [15]:
%%time

from sagemaker.pytorch import PyTorchModel

# Create a model from the trained estimator data and point to the prediction script
model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='1.4.0',
                     py_version='py3',
                     entry_point='predict.py',
                     source_dir='source_pytorch')

# deploy the model
predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-------------------!CPU times: user 558 ms, sys: 20.7 ms, total: 579 ms
Wall time: 9min 33s


# Deploy Classifier-2

In [23]:
%%time 
# deploy and create a predictor
linear_predictor = linear.deploy(initial_instance_count=1, 
                              instance_type='ml.t2.medium')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-----------------!CPU times: user 336 ms, sys: 49 µs, total: 336 ms
Wall time: 8min 32s


# Deploy Classifier-3

In [48]:
%%time 
# deploy and create a predictor
nb_predictor = sklearn_estimator.deploy(initial_instance_count=1, 
                              instance_type='ml.t2.medium')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


---------------!CPU times: user 271 ms, sys: 18.6 ms, total: 289 ms
Wall time: 7min 32s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to our test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

In [49]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

## EXERCISE: Determine the accuracy of your model

Use your deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. You may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

**To pass this project, your model should get at least 90% test accuracy.**

In [50]:
# First: generate predicted, class labels
import numpy as np

### Classifier-1 Results ###
# test_y_preds = np.squeeze(np.round(predictor.predict(test_x.values)))
### Classifier-1 Results ###

### Classifier-2 Results ### Best Model
result = linear_predictor.predict(test_x.values.astype('float32')) #returns a list of Records
test_y_preds = [val.label['predicted_label'].float32_tensor.values[0] for val in result]
### Classifier-2 Results ###

### Classifier-3 Results ###
# test_y_preds = np.squeeze(np.round(nb_predictor.predict(test_x.values)))
### Classifier-3 Results ###

test_y_preds = [int(value) for value in test_y_preds]
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [42]:
# Second: calculate the test accuracy
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, test_y_preds)

print(accuracy)

## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

0.96

Predicted class labels: 
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


# Prediction Test Results for all Classifiers

In [53]:
#Neural Network Model, 80% Accuracy
classifier1_test_predictions = [1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0] 

In [54]:
#LinearLearner, 96% Accuracy, Best Model
classifier2_test_predictions = [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0] 

In [55]:
#Naive Bayes Model, 92% Accuracy
classifier3_test_predictions = [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1] 

# Tabular Results Summary

<a id='table'></a>

| Classifier | Training Time | Accuracy |
| --- | --- | --- |
| Feedforward Neural Network | 67 seconds | .80 | 
| LinearLearner | 57 seconds | .96 | 
| Naive Bayes | 55 seconds | .92 | 

### Question 1: How many false positives and false negatives did your model produce, if any? And why do you think this is?

In [99]:
def compute_values(y_actual, y_pred):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_pred)): 
        if y_actual[i]==y_pred[i]==1:
           TP += 1
        if y_pred[i]==1 and y_actual[i]==0:
           FP += 1
           print ('FP for document in index {} \n'.format(i))
        if y_actual[i]==y_pred[i]==0:
           TN += 1
        if y_pred[i]==0 and y_actual[i]==1:
           FN += 1
           print ('FN for document in index {} \n'.format(i))

    return(TP, FP, FN, TN)

values = compute_values(test_y, classifier2_test_predictions) #use test prediction results of the best classifier

tp, fp, fn, tn = values[0], values[1], values[2], values[3]

print('TP: {}, FP: {}, FN: {}, TN: {}'.format(tp, fp, fn, tn))

FP for document in index 20 

TP: 15, FP: 1, FN: 0, TN: 9


** Answer**: Out of the 25 test predictions, the best classifier LinearLearner generated 1 fase positive (FP) and 0 false negatives (FN), as shown above. The binary classifier was able to learn the different types of plagiarism present in the train documents, which resulted in 0 FN in the test prediction results. Only the test document at the dataframe index 20 was misclassified by the model as plagiarized, as shown in the cell below. One reason for this might be because the 1-gram containment feature value of that test document was quite high (0.613924) while the 1-gram containment feature values of the training documents, which the model was trained on, were mostly in the range from 0.3 to 0.5, as shown in the second cell below.

In [91]:
display(test_data.loc[20, :]) #misclassified document info

0    0.000000
1    0.613924
2    0.000000
3    0.298343
Name: 20, dtype: float64

In [90]:
train_data = pd.read_csv(os.path.join(data_dir, "train.csv"), header=None, names=None)
display(train_data[train_data.iloc[:, 0]==0]) #feature values for train documents with class label 0

,0,1,2,3
0,0,0.398148,0.000000,0.191781
3,0,0.544503,0.000000,0.242574
4,0,0.329502,0.000000,0.161172
5,0,0.590308,0.000000,0.301653
7,0,0.516129,0.000000,0.270833
8,0,0.440860,0.000000,0.223958
13,0,0.441176,0.000000,0.305556
14,0,0.488889,0.000000,0.282609
16,0,0.611111,0.000000,0.324675
21,0,0.420382,0.000000,0.218750


### Question 2: How did you decide on the type of model to use? 

** Answer**: For the binary classification problem of plagiarism detection, I decided on what learning algorithm to use based on the data distribution and features available. Since the data set is small with less than 100 training documents that are fairly distributed over the 2 classes and contains small feature space per sample, my intuition was that simplest models, like Linear and Naive Bayes classifiers, would give the best results as there isn't enough data to estimate many parameters in more complex classifiers like Neural Networks. So I trained 3 different classifiers on the same training data - (1) Feed Forward Neural Network Model, (2) LinearLearner Model, and (3) Naive Bayes Model. The evaluation results on the test data are shown in the summary [table](#table) above. The LinearLearner gave the best results, with an accuary of 96%, which means that the learned model is able to separate the two classes of data and effectively output labels as plagiarized or non-plagiarized.   

----
## EXERCISE: Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`. You need to show, in this notebook, that the endpoint was deleted. Any other resources, you may delete from the AWS console, and you will find more instructions on cleaning up all your resources, below.

In [97]:
# uncomment and fill in the line below!
linear_predictor.delete_endpoint()

### Deleting S3 bucket

When you are *completely* done with training and testing models, you can also delete your entire S3 bucket. If you do this before you are done training your model, you'll have to recreate your S3 bucket and upload your training data again.

In [98]:
# deleting bucket, uncomment lines below

bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '6EF731FE163B0C5D',
   'HostId': 'Ol/lMRX0i6/jmiK76LOUKqtLwpBZcu0C2WwEhPISPG164CjFHcoFEITbe3um/zwKt9nPntlqmic=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'Ol/lMRX0i6/jmiK76LOUKqtLwpBZcu0C2WwEhPISPG164CjFHcoFEITbe3um/zwKt9nPntlqmic=',
    'x-amz-request-id': '6EF731FE163B0C5D',
    'date': 'Fri, 04 Dec 2020 00:54:00 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'plagiarism-detection/test.csv'},
   {'Key': 'plagiarism-detection-nb/sagemaker-scikit-learn-2020-12-03-20-42-47-093/output/model.tar.gz'},
   {'Key': 'sagemaker-record-sets/LinearLearner-2020-12-03-19-01-13-845/.amazon.manifest'},
   {'Key': 'sagemaker-scikit-learn-2020-12-03-20-20-38-669/source/sourcedir.tar.gz'},
   {'Key': 'sagemaker-record-sets/LinearLearner-2020-12-03-19-01-13-845/matrix_0.pbr'},
   {'Key': 'plagiarism-detection-linearlea

### Deleting all your models and instances

When you are _completely_ done with this project and do **not** ever want to revisit this notebook, you can choose to delete all of your SageMaker notebook instances and models by following [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Before you delete this notebook instance, I recommend at least downloading a copy and saving it, locally.

---
## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.

These are all just options for extending your work. If you've completed all the exercises in this notebook, you've completed a real-world application, and can proceed to submit your project. Great job!